# DWD_historical_weather: Beispiel-Notebook

#### Bundesland als globalen Parameter festlegen.

In [ ]:
BUNDESLAND = 'Berlin'

In [ ]:
from dwd_hist_weather import tagestemp_land

import pandas as pd
import pickle

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns

#### Das eigentliche Einlesen der Daten: Wenn vorhanden aus pickle, sonst **tagestemp_land** aus dem Modul aufrufen und die Daten vom DWD ziehen.

In [ ]:
pickle_dateiname = f'{BUNDESLAND}.pickle'
try:
    tageswerte = pickle.load(open(pickle_dateiname, 'rb'))
    print(f'Wetterdaten für {BUNDESLAND} aus pickle eingelesen.')
except (OSError, IOError):
    tageswerte = tagestemp_land(BUNDESLAND)
    pickle.dump(tageswerte, open(pickle_dateiname, 'wb'))
    print(f'\nWetterdaten für {BUNDESLAND} in pickle geschrieben.')

### DataFrame ausgeben

In [ ]:
display(tageswerte)

### Heatmap der täglichen Durchschnittstemperaturen

In [ ]:
ana = tageswerte.pivot(index='Jahr', columns='Tag_des_Jahres', values='Temp')

f, ax = plt.subplots(figsize=(20, 10))
sns.heatmap(ana, vmin=-10, vmax=23, cmap="RdBu_r")
ax.axes.set_title("Tagesmitteltemperaturen", y=1.01)
ax.xaxis.set_major_locator(mdates.MonthLocator())
ax.xaxis.set_minor_locator(mdates.DayLocator())
ax.xaxis.set_major_formatter(mdates.DateFormatter('%b'))

### Jährliche Durchschnittstemperaturen plus 5-Jahres-Mittel

In [ ]:
ana = tageswerte.pivot(index='Jahr', columns='Tag_des_Jahres', values='Temp')

ana['Jahresmittel'] = ana.mean(axis=1)
ana['Jahresmittel5'] = ana['Jahresmittel'].rolling(5).mean()

plt.subplots(figsize=(20, 10))
sns.lineplot(data=ana, x='Jahr', y='Jahresmittel')
sns.lineplot(data=ana, x='Jahr', y='Jahresmittel5', color='red')